## Notebook responsável pela geração do mapa utilizado pelo SVAA

In [ ]:
import os

import folium
import numpy as np
import pandas as pd
import requests as req
from branca.element import MacroElement
from folium.plugins import Fullscreen
from jinja2 import Template
from resolve_path import ajuste_path, read_input

### Carrega Datasets

In [ ]:
pathUtil = ajuste_path('data/util/')

df_lt = read_input('ESUL-LIs-LTs e vaos torres.csv')

df_mapa = pd.read_csv(pathUtil + 'dataset_mapa_mockado.csv', sep=',')

## Preparação do Mapa

### Pré-processamento do dataset de Linhas de Tramissão

In [ ]:
# Remove linhas com valores missing de latitude e/ou longitude
df_lt_processado = df_lt.dropna(subset=['Latitude', 'Longitude']).copy()

# Converte as colunas latitude e longitude para float
df_lt_processado.loc[:, ['Latitude', 'Longitude']] = df_lt_processado.loc[:, [
    'Latitude', 'Longitude']].replace(',', '.', regex=True)
df_lt_processado['Latitude'] = pd.to_numeric(
    df_lt_processado['Latitude'], errors='coerce')
df_lt_processado['Longitude'] = pd.to_numeric(
    df_lt_processado['Longitude'], errors='coerce')

### Preparação do df_lt

In [ ]:
# Filtra o df_lt para conter apenas Linhas de Transmissão que estão contidas no df_mapa
def filtra_linhas_transmissao(df_lt, df_mapa):
    # Cria uma lista com os locais de instalação do df_mapa
    lista_locais = df_mapa['local_de_instalacao'].unique()
    # Filtra o df_lt para conter apenas Linhas de Transmissão que estão contidas no df_mapa
    lts_contida_em_ambos_datasets = df_lt['local_prefixo'].isin(lista_locais)

    df_lt_filtrado = df_lt[lts_contida_em_ambos_datasets]

    return df_lt_filtrado

# Adiciona uma coluna com o prefixo de cada linha de transmissão (ex: 'S-L-ARE/CBA-C1-LT525-V0372' -> 'S-L-ARE/CBA')
df_lt_processado['local_prefixo'] = df_lt_processado['Local de instalação'].str[:11]

df_lt_filtrado = filtra_linhas_transmissao(df_lt_processado, df_mapa)


print("Número de linhas após a filtragem: " + str(df_lt_filtrado.shape[0]))

### Definição das cores que serão utilizadas no mapa 

In [ ]:
corMaisFraca = "#dfc27d"
corMediaFraca = "#bf812d"
corMediaForte = "#8c510a"
corMaisForte = "#543105"


cores_intermediarias_gradiente = [
    corMaisFraca, corMediaFraca, corMediaForte, corMaisForte]


def get_color_discrete(score):
    if score is np.nan:
        return "#000000"
    elif score < 0.25:
        return corMaisFraca
    elif score < 0.5:
        return corMediaFraca
    elif score < 0.75:
        return corMediaForte
    else:
        return corMaisForte

### Carrega atributos necessários para a exibição do mapa

In [ ]:
# calcula onde vai começar centralizado o mapa
media_x = df_mapa["latitude"].mean()
media_y = df_mapa["longitude"].mean()

# carrega os polígonos dos Estados do Brasil
url_poly_brasil = "https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=UF"

headers = {"Accept": "application/vnd.geo+json"}

mapa_brasil = req.get(url_poly_brasil, headers=headers)

poly_brasil = mapa_brasil.json()

### Funções utilizadas pelo mapa

In [ ]:
def adiciona_camadas(map_object, df, nome_camada):
    '''
    Função que adiciona camadas ao mapa.

    Parâmetros:
    - map_object: objeto folium.Map
        O objeto do mapa ao qual as camadas serão adicionadas.
    - df: pandas.DataFrame
        DataFrame contendo os dados que serão usados para adicionar as camadas.
        Deve conter as colunas 'latitude', 'longitude', 'local_de_instalacao', 'mes', 'ano', 'hh_total' e 'probabilidade'.
    - nome_camada: str
        O nome da camada que será adicionada ao mapa.

    Adiciona um marcador ao mapa para cada local de instalação no DataFrame data.
    Se for uma linha de transmissão, o marcador será um losangulo, se não será um circulo.
    '''
    camada = folium.FeatureGroup(name=nome_camada)
    for _, row in df.iterrows():
        color = get_color_discrete(row["probabilidade"])

        # Altera a cor da fonte do popup de acordo com a cor do marcador
        font_color = '#543105'
        if color in ["#543105", "#8c510a"]:
            font_color = '#dfc27d'
        else:
            if color in ["#dfc27d"]:
                font_color = '#8c510a'
        popup_content = f"""
            <div>
                <div style="font-size: 1.1em;">
                    <b>Local:</b> {row['local_de_instalacao']}<br>
                </div>
                <div style="margin-top: 10px;">
                    <b>HH total por mês em {(row['mes'])}/{(row['ano'])}:</b> {round(row['hh_total'], 2)}<br>
                    <b>Probabilidade:</b> {round(row['probabilidade'], 2)}
                </div>
            </div>
        """
        if row['local_de_instalacao'].startswith('S-L'):  # Se for uma linha de transmissão
            html = f"""
                <div style="
                    background-color: {color};
                    color: {font_color};
                    border-radius: 50%;
                    width: 40px;
                    height: 40px;
                    display: flex;
                    justify-content: center;
                    align-items: center;
                    font-size: 10px;
                    font-weight: bold;
                    transform: translate(-50%, -50%);
                    clip-path: polygon(50% 0%, 100% 50%, 50% 100%, 0% 50%);
                ">
                    {round(row['probabilidade'] * 100, 2)}%
                </div>
            """
        else:  # Se for uma subestação, desenha um círculo
            html = f"""
                <div style="
                    background-color: {color};
                    color: {font_color};
                    width: 40px;
                    height: 40px;
                    border-radius: 50%;
                    display: flex;
                    justify-content: center;
                    align-items: center;
                    font-size: 10px;
                    font-weight: bold;
                    transform: translate(-50%, -50%);
                ">
                    {round(row['probabilidade'] * 100, 2)}%
                </div>
            """

        # Adiciona o conteúdo ao mapa
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.DivIcon(html=html),
            popup=folium.Popup(popup_content, max_width=2650),
            zIndexOffset=round(row["probabilidade"] * 100),
            tooltip=row['local_de_instalacao']
        ).add_to(camada)

        map_object.add_child(camada)


# Função que define o período de tempo para a visualização do mapa
def filtra_df_por_tempo(df, mes, ano):
    df = df.query("`mes` == @mes and `ano` == @ano")
    return df


def adiciona_vaos_lt(map_object, df_linhas_de_transmissao, cor):
    '''
    Função que adiciona os vãos de cada linha de transmissão ao mapa.
    '''
    lista_coordenadas = []
    camada = folium.FeatureGroup(name='Linhas de Transmissão')
    for group, lt in df_linhas_de_transmissao.groupby('local_prefixo'):
        lt = lt.reset_index(drop=True)
        for _, row in lt.iterrows():
            lista_coordenadas.append((row['Latitude'], row['Longitude']))

        folium.PolyLine(
            lista_coordenadas,
            color=cor,
            weight=4,
            opacity=0.7,
            tooltip=group,
            no_clip=True,
        ).add_to(camada)

        lista_coordenadas = []

    map_object.add_child(camada)

### Define classe para legenda utilizada no mapa

In [ ]:
# Defina a classe Legend (Para que a legenda não desapareça no modo fullscreen)


class Legend(MacroElement):
    def __init__(self, legend_html, position='bottomright'):
        super(Legend, self).__init__()
        self._name = 'Legend'
        self.legend_html = legend_html
        self.position = position
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            L.Control.Legend = L.Control.extend({
                onAdd: function(map) {
                    var div = L.DomUtil.create('div', '');
                    div.innerHTML = `{{this.legend_html}}`;
                    return div;
                },
                onRemove: function(map) {
                    // Nada a fazer aqui
                }
            });
            L.control.legend = function(opts) {
                return new L.Control.Legend(opts);
            }
            L.control.legend({ position: '{{this.position}}' }).addTo({{this._parent.get_name()}});
        {% endmacro %}
        """)

## Criação do Mapa

In [ ]:
# Criar o mapa centrado nas coordenadas especificadas
mapa = folium.Map(
    location=[media_x, media_y],
    zoom_start=5,
    control_scale=True,
    dragging=True,
    max_zoom=18,
    min_zoom=4,

)

# Adiciona o botão de fullscreen ao mapa
Fullscreen(position='topright', force_separate_button=True).add_to(mapa)

# Adiciona html de legenda ao mapa
caminho_legenda_html = ajuste_path('src/core/mapa/assets/legenda_mapa.html')
with open(caminho_legenda_html, "r", encoding="utf-8") as arquivo:
    legenda_html = arquivo.read()

legend_css = '''
<div style="
    background-color: rgba(255, 255, 255, 0.8);
    padding: 10px;
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.4);
    ">
    {conteudo}
</div>
'''.format(conteudo=legenda_html)

legend = Legend(legend_html=legenda_html, position='bottomright')
mapa.add_child(legend)

# Adicionar a camada de polígonos de estados do Brasil
camada_poligonos_brasil = folium.FeatureGroup(name="Polígonos do Brasil")
folium.GeoJson(
    poly_brasil,
    zoom_on_click=True,
    style_function=lambda feature: {
        "fillColor": "#c8d977",
        "color": "#A9A9A9",
        "weight": 2,
        "dashArray": "5, 5",
        "fillOpacity": 0.15,
    },
).add_to(camada_poligonos_brasil)
mapa.add_child(camada_poligonos_brasil)

# Filtra o df_mapa somente com os valores de probabilidade máxima para cada local, ano e mês.
df_mapa_max_prob = df_mapa.loc[df_mapa.groupby(
    ['ano', 'mes', 'local_de_instalacao'])['probabilidade'].idxmax()]

# Adicionar camadas de visualização ao mapa
adiciona_camadas(mapa, filtra_df_por_tempo(df_mapa_max_prob, 2, 2024,),
                 "Exibir probabilidade de 02/2024")


# ADICIONA ACIDENTES AO MAPA

# Filtrar o DataFrame para remover linhas com valores NaN nas colunas de latitude e longitude
df_acidentes = df_mapa.dropna(
    subset=['latitude_acidente', 'longitude_acidente'])

# Adicionar marcadores ao mapa para os acidentes
for _, row in df_acidentes.iterrows():
    if row['potencial'] <= 8:
        color = '#d7301f'
    elif row['potencial'] <= 12:
        color = '#fec44f'
    else:
        color = '#006d2c'
    folium.Marker(
        location=[row['latitude_acidente'], row['longitude_acidente']],
        popup=f"<b>Potencial: {row['potencial']}",
        rise_on_hover=True,
        zIndexOffset=1,
        icon=folium.DivIcon(
            html=f"""
        <div style="
            width: 0;
            height: 0;
            border-left: 12px solid transparent;
            border-right: 12px solid transparent;
            border-bottom: 24px solid {color};
            position: relative;
        ">
            <div style="
                position: absolute;
                top: 5px;  # Ajuste a posição vertical do texto
                left: 50%;
                transform: translate(-50%, 0);
                color: white;
                font-size: 14px;
                font-weight: bold;
            ">
                !
            </div>
        </div>
        """
        )
    ).add_to(mapa)

    # Adiciona os vãos das Linhas de Tranmissão (LT) ao mapa
    adiciona_vaos_lt(mapa, df_lt_filtrado, '#525252')

    # Adiciona a camada de controle de camadas ao mapa
    folium.LayerControl().add_to(mapa)

### Salva o mapa dentro do folder 'mapas_html'

In [ ]:
# Gerar o HTML do mapa como uma string
mapa_html = mapa._repr_html_()

# Salvar a string em um arquivo com encoding UTF-8
caminho_assets = ajuste_path('src/core/mapa/assets/mapa.html')

with open(caminho_assets, "w", encoding="utf-8") as arquivo:
    arquivo.write(mapa_html)

# Exibe o mapa
mapa